<a href="https://colab.research.google.com/github/fizznix/Llama3-hf/blob/main/LLama_3_huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install libraries**



In [ ]:
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
import json
import torch
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          pipeline)

**HF account config**

In [ ]:
config_data = json.load(open('config.json'))
HF_TOKEN = config_data['HF_TOKEN']

In [ ]:
model_name = "meta-llama/Meta-Llama-3-8B"

**Quantisation configuration**

Purpose - It shrinks the model
- changes the precision of the weights
- models might be of higher precision 32 bits we can convert it to 4/8 bits

Trade off -
Loss in precision

In [ ]:
# bnb_4bit_use_double_quant helps in keeping the precision
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
    )

**Load tokenizer and LLM**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          token=HF_TOKEN)
# padding token and eos_token = end of sequence token
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# quantization_config is used so that we compress the model else the whole model gets downloaded
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    token=HF_TOKEN
    )

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [ ]:
# https://huggingface.co/docs/transformers/en/main_classes/pipelines
# max_new_tokens = length of expected output default 20 tokens
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128
)

In [ ]:
def get_response(prompt):
  sequences = text_generator(prompt)
  gen_text = sequences[0]["generated_text"]
  return gen_text

In [ ]:
prompt = "What are tokenizers?"
get_response(prompt)

'What are tokenizers? Tokenizers are used to split text into words, punctuation, numbers, etc. They are commonly used to prepare text data for machine learning models. They are commonly used to prepare text data for machine learning models. This is a very common task in natural language processing (NLP) applications. Tokenizers can be used to split text into words, punctuation, numbers, etc. They are commonly used to prepare text data for machine learning models. They are commonly used to prepare text data for machine learning models. This is a very common task in natural language processing (NLP) applications. Tokenizers can be used to split text into words,'